In [ ]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.7 MB/s 
     |████████████████████████████████| 8.2 MB 11.0 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=1d034c2d0947610f0a1bd73ce19c613bee3b7f11ccdc9e8d3e0aff9ce593399c
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt


In [ ]:
import pandas as pd

Отзывы для обучения сохранил в файл. Считываем

In [ ]:
revs = pd.read_csv('/content/revs.csv')
revs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   rev     60 non-null     object
 1   val     60 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ KB


In [10]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('stopwords')

sw = stopwords.words('russian')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [11]:
from pymorphy2 import MorphAnalyzer

morph = MorphAnalyzer()

In [ ]:
from collections import Counter

Функция получает из текстов отзывов список лемм без стоп-слов и слов, встретившихся менее 2 раз

In [12]:
def prepare_revs(revs):
    tokens = [token for token in word_tokenize(revs)]
    tokens_lower = [token.lower() for token in  tokens]
    tokens_filtered = [token for token in tokens_lower if token not in sw]
    lemmas = [morph.parse(token)[0].normal_form for token in tokens_filtered]
    lemmas_counter = Counter(lemmas)
    lemmas_filtered = [lemma for lemma in lemmas_counter.keys() if lemmas_counter[lemma] > 1]
    return set(lemmas_filtered)

Готовим множества лемм-маркеров положительных и отрицательных отзывов

In [14]:
positive_revs = ' '.join(revs[revs['val'] == 1]['rev'])
positive_lemmas_set = prepare_revs(positive_revs)

In [15]:
negative_revs = ' '.join(revs[revs['val'] == 0]['rev'])
negative_lemmas_set = prepare_revs(negative_revs)

In [16]:
positive_markers = positive_lemmas_set - negative_lemmas_set
negative_markers = negative_lemmas_set - positive_lemmas_set

Эта функция нужна для предсказания по отзову его полярности и предобрабатывает его, превращая в список лемм

In [20]:
def prepare_a_rev(rev):
    tokens = [token for token in word_tokenize(rev)]
    tokens_lower = [token.lower() for token in  tokens]
    lemmas = [morph.parse(token)[0].normal_form for token in tokens_lower]
    return lemmas

In [21]:
import random

Собственно функция предсказания полярности отзыва

In [22]:
def predict(rev, positive_markers, negative_markers):
    rev_lemmas = prepare_a_rev(rev)
    pos = 0
    neg = 0
    for lemma in rev_lemmas:
        if lemma in positive_markers:
            pos += 1
        elif lemma in negative_markers:
            neg += 1
    if pos > neg:
        return 1
    elif pos < neg:
        return 0
    else:
        return random.choice([0, 1])

Подсчет accuracy на тестовых данных

In [23]:
test_data = pd.read_csv('/content/revs_test.csv')
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   rev     10 non-null     object
 1   val     10 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 288.0+ bytes


In [24]:
x = test_data['rev']
y = test_data['val']

In [25]:
y_pred = [predict(rev, positive_markers, negative_markers) for rev in x]

In [26]:
from sklearn.metrics import accuracy_score

In [28]:
accuracy_score(y, y_pred)

0.9